In [1]:
# Import Pandas
import pandas as pd

#Set Relative Path
import sys
sys.path.append("../src")

In [2]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall.csv')
df_dataK.head()

,text,emotion
0,i got up feeling horny this morning,love
1,i will go to my mailbox and talk to the mailma...,happy
2,i understand that any of my extremely positive...,happy
3,i suggest you give it a listen i feel like i a...,love
4,i have been with petronas for years i feel tha...,happy


In [3]:
from nltk.stem import WordNetLemmatizer
def lemmatizer(text):
    """Apply Wordnet lemmatizer to text (go to root word)"""
    wnl = WordNetLemmatizer()
    text = [wnl.lemmatize(word) for word in text.split()]
    return " ".join(text)
df_dataK['text'] = df_dataK['text'].apply(lemmatizer)

In [4]:
# suppression des caratères spéciaux, formatage des contractions...
from nltk.corpus import stopwords
from emoji import demojize
import re

def clean_str(texts):
    from nltk.corpus import stopwords
    # Lowercasing
    texts = texts.str.lower()

    # Remove special chars
    texts = texts.str.replace(r"(http|@)\S+", "")
    texts = texts.apply(demojize)
    texts = texts.str.replace(r"::", ": :")
    texts = texts.str.replace(r"’", "'")
    texts = texts.str.replace(r"[^a-z\':_]", " ")

    # Remove repetitions
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    texts = texts.str.replace(pattern, r"\1")

    # Transform short negation form
    texts = texts.str.replace(r"(can't|cannot)", 'can not')
    texts = texts.str.replace(r"(ain't|wasn't|weren't)", 'be not')
    texts = texts.str.replace(r"(don't|didn't|didnt)", 'do not')
    texts = texts.str.replace(r"(haven't|hasn't)", 'have not')
    texts = texts.str.replace(r"(won't)", 'will not')
    texts = texts.str.replace(r"(im)", ' i am')
    texts = texts.str.replace(r"(ive)", ' i have')
    texts = texts.str.replace(r"(n't)", ' not')

    # Remove stop words
    stopwords = stopwords.words('english')
    stopwords.remove('not')
    stopwords.remove('nor')
    stopwords.remove('no')
    texts = texts.apply(lambda x: ' '.join([word for word in x.split() if (word not in stopwords and len(word) > 1 )]))
    return texts

df_dataK['text'] = clean_str(df_dataK['text'])

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sentence_transformers import SentenceTransformer
def model_used(df, model):
    """Given a model choice, return the model and the computed matrix"""
    if model == 'Tfidf':
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
        tfidf_matrix = tf.fit_transform(df['text'])
        return tf, tfidf_matrix
    elif model == 'CountVectorizer':
        cv = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
        matrix = cv.fit_transform(df['text'])
        return cv, matrix
    elif model == 'BERT':
#         bert = SentenceTransformer('distiluse-base-multilingual-cased-v1') # Multilingue
#         bert = SentenceTransformer('average_word_embeddings_glove.6B.300d') # + rapide
        bert = SentenceTransformer('paraphrase-MiniLM-L6-v2') # Meilleur score en théorie, à vérifier sur nos données
        matrix = bert.encode(df['text'].astype('str'), show_progress_bar=True)
        return bert, matrix

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder

# Set X and y
enc, X = model_used(df_dataK,'BERT')

Labelenc = LabelEncoder()
df_dataK['emotion'] = Labelenc.fit_transform(df_dataK['emotion'])
y = df_dataK['emotion']

You try to use a model that was created with version 1.2.0, however, your version is 1.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [7]:
from reg_lin import get_metrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


In [9]:
from reg_lin import get_metrix, get_metrix_v2
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential 
from tensorflow.keras import layers, preprocessing
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score


models = [  LogisticRegression(n_jobs=-1,multi_class='multinomial',penalty='l2',solver = 'saga' , max_iter=500,C=1 ),
            SGDClassifier(n_jobs=-1,penalty='l2', max_iter=500,loss='hinge',alpha=0.001),
            RandomForestClassifier(n_jobs=-1,n_estimators= 1000,min_samples_split= 8,min_samples_leaf= 2,criterion= 'entropy')]
#             XGBClassifier(),
#             Sequential()]

place = 4
savedmodel=[]
count = 0
results= {}
for modelset in models:
    if 'Sequential' in str(modelset):
        df_dataKKeras = df_dataK.sample(15000)
        y = df_dataKKeras['emotion']

        df_dataKKeras = df_dataKKeras[['text']]
        tk = preprocessing.text.Tokenizer()
        tk.fit_on_texts(df_dataKKeras['text'])
        X = tk.texts_to_matrix(df_dataKKeras['text'], mode='tfidf')

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=None)
        model = modelset
        model.add(layers.Dense(160, activation='relu', input_shape=[X_train.shape[1]]))
        model.add(layers.Dense(80, activation='relu'))
        model.add(layers.Dense(40, activation='relu'))
        model.add(layers.Dense(6, activation='softmax'))
        model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=10, batch_size=100)

        y_train_pred = model.predict_classes(X_train)
        y_test_pred = model.predict_classes(X_test)
        y_test_proba = model.predict_proba(X_test)
        y_test_pred = Labelenc.inverse_transform([round(y_test_pred[0][0])])
        print(" Train accuracy = {}"
              .format(round(accuracy_score(y_train, y_train_pred), 3)))

        print(" Test accuracy = {}"
              .format(round(accuracy_score(y_test, y_test_pred), 3)))
        results['NeuralNetwork'] = [model, y_test_proba, y_test_pred, y_test]
    else:
        try:
            model, y_test_proba, y_test_pred, y_test = get_metrix_v2(y, X, modelset)
        except:
            model, y_test_pred, y_test = get_metrix(y, X, modelset)
            y_test_proba = None
        y_test_pred = Labelenc.inverse_transform(y_test_pred)
        if 'Logistic' in str(modelset):
            results['LogisticRegression'] = [model, y_test_proba, y_test_pred, y_test]
        elif 'SGD' in str(modelset):
            results['SGDClassifier'] = [model, y_test_proba, y_test_pred, y_test]
        elif 'Forest' in str(modelset):
            results['RandomForest'] = [model, y_test_proba, y_test_pred, y_test]
        elif 'XGB' in str(modelset):
            results['Xgboost'] = [model, y_test_proba, y_test_pred, y_test]
        else:
            results[str(model)] = [model, y_test_proba, y_test_pred, y_test]


/home/apprenant/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Train R2 = -0.29; Train RMSE 1.589
Test R2  = -0.35; Test RMSE 1.61
Train R2 = -0.417; Train RMSE 1.665
Test R2  = -0.46; Test RMSE 1.681
Train R2 = 0.981; Train RMSE 0.191
Test R2  = -0.33; Test RMSE 1.614


In [10]:
results

{'LogisticRegression': [LogisticRegression(C=1, max_iter=500, multi_class='multinomial', n_jobs=-1,
                     solver='saga'),
  array([[2.79707145e-02, 1.59962565e-01, 2.47402722e-03, 1.89044874e-03,
          8.07208240e-01, 4.94063890e-04],
         [1.17768683e-02, 3.54708910e-01, 8.21517184e-02, 1.85649972e-02,
          4.63497519e-01, 6.92999586e-02],
         [1.83975045e-02, 1.06561504e-01, 6.05407774e-01, 8.82487670e-02,
          1.00317955e-01, 8.10665861e-02],
         ...,
         [6.81919104e-04, 8.12693238e-02, 7.08899796e-01, 1.47259817e-01,
          4.89583649e-02, 1.29307508e-02],
         [1.23182163e-02, 6.89486265e-02, 2.27198750e-01, 6.56326354e-01,
          1.75954625e-02, 1.76126696e-02],
         [7.23050356e-01, 3.35451402e-02, 8.10149033e-03, 9.34884045e-03,
          2.19858065e-01, 6.09621825e-03]], dtype=float32),
  array(['sadness', 'sadness', 'happy', ..., 'happy', 'love', 'anger'],
        dtype=object),
  10936    1
  30676    4
  36012  

In [11]:
import joblib

In [12]:
joblib.dump(results,"../dump/results_multi.joblib", compress=3)

['../dump/results_multi.joblib']

In [17]:
inverser = Labelenc.inverse_transform([0,1,2,3,4,5])

In [18]:
inverser

array(['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
      dtype=object)

In [14]:
import random as rd

In [16]:
encoded = [rd.randint(0,5) for x in range(50)]

In [19]:
revencoded = [inverser[x] for x in encoded]

In [20]:
revencoded

['anger',
 'sadness',
 'love',
 'fear',
 'love',
 'happy',
 'anger',
 'sadness',
 'love',
 'love',
 'love',
 'love',
 'surprise',
 'happy',
 'anger',
 'anger',
 'happy',
 'surprise',
 'happy',
 'happy',
 'love',
 'happy',
 'fear',
 'happy',
 'anger',
 'fear',
 'fear',
 'happy',
 'surprise',
 'love',
 'surprise',
 'anger',
 'happy',
 'love',
 'love',
 'sadness',
 'surprise',
 'surprise',
 'sadness',
 'love',
 'happy',
 'love',
 'sadness',
 'happy',
 'love',
 'happy',
 'sadness',
 'anger',
 'sadness',
 'anger']

In [ ]:
del x,X,y,X_test,X_train,y_test,y_train,y_test_pred,y_train_pred,df_dataK,df_dataKKeras

In [ ]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall_test.csv')

df_dataK['text'] = df_dataK['text'].apply(lemmatizer)
df_dataK['text'] = clean_str(df_dataK['text'])
X_test = enc.encode(df_dataK['text'])
y_test = Labelenc.transform(df_dataK['emotion'])

count = 0
place = 4 

for model in savedmodel:

    print (model)
    if count != place:
        y_test_pred = model.predict(X_test)
    if count == place:
        X_test = tk.texts_to_matrix(df_dataK['text'], mode='tfidf')
        y_test_pred = model.predict(X_test)
    print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))

    count += 1
